In [1]:
from tqdm import tqdm
from itertools import product
from glob import glob

import pandas as pd

import torch

from discovery_utils import auto_circuit

from experiments import benchmark_experiment
from utils import get_data, load_gpt2_tl

torch.set_grad_enabled(False)

device = "cuda" if torch.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2

/home/jgcarrasco/.virtualenvs/mech_interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task = "greater-than"
ablation_scheme = "zero"

In [3]:
if task == "greater-than":
        thresholds = 10**torch.linspace(-5, 1, 30)
else:
    thresholds = 10**torch.linspace(-5, 0, 30)
if task == "ioi":
    n_patching = 150
    n_val = 150
else:
    n_patching = 250
    n_val = 250

rows = []

data = get_data(n_patching=n_patching, n_val=n_val, task=task)

patching_tokens = data["patching_tokens"].cuda()
patching_cache = data["patching_cache"]

val_tokens = data["val_tokens"].cuda()
val_answer_tokens = data["val_answer_tokens"].cuda()
val_logits = data["val_logits"]

gt_heads = data["gt_circuit"]

for threshold in thresholds:
    model = load_gpt2_tl()
    circuit_heads, _ = auto_circuit(
    model, threshold, val_logits, val_tokens, patching_cache, 
    ablation_scheme=ablation_scheme, include_mlps=False)
    del model
    tpr = len([head for head in circuit_heads if head in gt_heads]) / len(gt_heads)
    fpr = len([head for head in circuit_heads if head not in gt_heads]) / (144 - len(gt_heads))

    rows.append([threshold.item(), tpr, fpr])

df = pd.DataFrame(rows, columns=["threshold", "TPR", "FPR"])

Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrai

In [4]:
try:
    max_i = max([int(path[-5]) for path in glob(f"roc_results/{task}_{ablation_scheme}_*.csv")])
except:
    max_i = 0

In [5]:
df.to_csv(f"roc_results/{task}_{ablation_scheme}_{max_i+1}.csv", index=False)